### Подготовка

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

from lets_plot import *
from lets_plot.mapping import as_discrete
LetsPlot.setup_html()

import os; import sys; sys.path.append(os.path.join(sys.path[0], "..", ".."))
from util import util

In [2]:
def plot_matrix(plots=[], width=400, height=300, columns=2):
    bunch = GGBunch()
    for i in range(len(plots)):
        row = int(i / columns)
        column = i % columns
        bunch.add_plot(plots[i], column * width, row * height, width, height)
    return bunch.show()

In [3]:
BASIC_FONT = ("Lucida Grande", 20, "")

In [4]:
char_widths_df = util.get_df("../../data/full/char_widths.csv", "train")
char_widths_df

,char_id,char,alphabet,font_face,font_size,font_version,width
0,65,A,basic_latin,Courier,9,,9
1,66,B,basic_latin,Courier,9,,9
2,67,C,basic_latin,Courier,9,,9
3,68,D,basic_latin,Courier,9,,9
4,69,E,basic_latin,Courier,9,,9
...,...,...,...,...,...,...,...
97939,38754,面,japanese,Verdana,20,bi,43
97940,38761,革,japanese,Verdana,20,bi,43
97941,38936,領,japanese,Verdana,20,bi,43
97942,39080,風,japanese,Verdana,20,bi,43


In [5]:
char_widths_s = char_widths_df.groupby(["font_face", "font_size", "font_version", "char"]).width.median().astype(int)
char_widths_s

font_face  font_size  font_version  char
Courier    9                                 9
                                    !        9
                                    "        9
                                    #        9
                                    $        9
                                            ..
Verdana    20         i             面       42
                                    革       42
                                    領       42
                                    風       42
                                    高       42
Name: width, Length: 90384, dtype: int64

## Шаг №1

Ширины символов пробразуются в "порядки": для каждого символа `X` каждого алфавита и для каждого шрифта `F` порядок - это количество символов этого же алфавита, которые по ширине меньше чем он сам, обозначаемое через `p(X, F)`. Средний порядок `p(X)` - это порядок, вычисляемый для ширин, усредненных по всем алфавитам и всем шрифтам `F`.

In [6]:
mean_order_df = util.transform_values_to_orders(
    char_widths_df[(char_widths_df.font_face == BASIC_FONT[0])&\
                   (char_widths_df.font_size == BASIC_FONT[1])&\
                   (char_widths_df.font_version == BASIC_FONT[2])],
    values_col="width",
    sorted_col="char"
).T.reset_index()
mean_order_df

,char,order
0,',0
1,l,0
2,i,0
3,j,3
4,ϊ,3
...,...,...
533,尊,318
534,高,318
535,W,535
536,Ю,535


## Шаг №2

Полученные значения `p(X)` складываем ровно в `carts_count` корзин.

In [7]:
carts_count = 10

In [8]:
mean_order_df["cart_id"] = KMeans(n_clusters=carts_count, random_state=42).fit(mean_order_df[["order"]]).labels_
cart_id_replaces = {
    k: i
    for i, k in enumerate(mean_order_df.groupby("cart_id")["order"].median().sort_values().keys())
}
mean_order_df = mean_order_df.replace({"cart_id": cart_id_replaces})
mean_order_df

,char,order,cart_id
0,',0,0
1,l,0,0
2,i,0,0
3,j,3,0
4,ϊ,3,0
...,...,...,...
533,尊,318,8
534,高,318,8
535,W,535,9
536,Ю,535,9


Рисуем `geom_boxplot()` с вариацией `p(X)` для каждой корзины:

In [9]:
ggplot(mean_order_df, aes(as_discrete("cart_id", order_by="..middle.."), "order")) + geom_boxplot()

Рисуем `geom_bar()` с `p(X)` для каждого `X` каждой корзины:

In [10]:
plot_matrix([
    ggplot(mean_order_df[mean_order_df.cart_id == cart_id]) + \
        geom_bar(aes(as_discrete("char", order_by="order", order=1), "order"), stat='identity') + \
        coord_flip() + ylim(0, 540) + \
        ggtitle("cart_id = {0}".format(cart_id))
    for cart_id in range(carts_count - 1, -1, -1)
])

## Шаг №3

Фиксируем следующие значения:

- `basic_cart_id` - индекс средней по ширине корзины.

- `basic_font` - "Lucida Grande", 20pt, без модификаций.

- `cart_widths` - словарь, в котором каждая корзина получает в качестве ширины медианное значение ширин (в пикселях) всех символов корзины, начертанных базовым шрифтом.

- `basic_width` - значение из `cart_widths`, принадлежащее корзине с индексом `basic_cart_id`.

In [11]:
cart_widths_df = pd.merge(char_widths_s.to_frame().reset_index(), mean_order_df, on="char").groupby(["font_face", "font_size", "font_version", "cart_id"]).width.median().reset_index(level=3)
cart_widths_df["font"] = ['/'.join([str(v) for v in t]) for t in cart_widths_df.index]
cart_widths_df

cart_id  width          font
font_face font_size font_version                              
Courier   9                             0    9.0    Courier/9/
                                        1    9.0    Courier/9/
                                        2    9.0    Courier/9/
                                        3    9.0    Courier/9/
                                        4    9.0    Courier/9/
...                                   ...    ...           ...
Verdana   20        i                   5   31.0  Verdana/20/i
                    i                   6   33.0  Verdana/20/i
                    i                   7   34.0  Verdana/20/i
                    i                   8   42.0  Verdana/20/i
                    i                   9   41.0  Verdana/20/i

[1680 rows x 3 columns]

In [12]:
basic_cart_id = int(carts_count / 2)
print("Basic cart_id:", basic_cart_id)
basic_font = BASIC_FONT
print("Basic font:", basic_font)
cart_widths = cart_widths_df.loc[basic_font].set_index("cart_id").width.astype(int).to_dict()
print("Cart widths:", cart_widths)
basic_width = cart_widths[basic_cart_id]
print("Basic width:", basic_width)

Basic cart_id: 5
Basic font: ('Lucida Grande', 20, '')
Cart widths: {0: 11, 1: 19, 2: 22, 3: 23, 4: 27, 5: 29, 6: 31, 7: 32, 8: 39, 9: 40}
Basic width: 29


In [13]:
ggplot(cart_widths_df.sort_values(by=["font_size", "font_face", "font_version"]), aes("font", "width")) + \
    geom_point(size=1) + geom_step(size=1) + \
    geom_point(data=cart_widths_df.loc[basic_font], \
               color="#de2d26", fill="#de2d26", shape=21, size=4, alpha=.5) + \
    facet_grid(y="cart_id") + \
    ggsize(1000, 3000)

## Шаг №4

Вычисляются нормировочные коэффициенты различных шрифтов. Для каждого `F` его нормировочный коэффициент `N(F)` - это отношение медианной ширины "базовой корзины" начертанной шрифтом `F` к `basic_width`.

In [14]:
ncoeff_s = char_widths_s.to_frame().reset_index().set_index("char").loc[mean_order_df[mean_order_df.cart_id == basic_cart_id].char.values].reset_index().groupby(["font_face", "font_size", "font_version"]).width.median() / basic_width
ncoeff_s

font_face  font_size  font_version
Courier    9                          0.379310
                      b               0.413793
                      bi              0.413793
                      i               0.379310
           11                         0.413793
                                        ...   
Verdana    17         i               0.931034
           20                         1.034483
                      b               1.137931
                      bi              1.137931
                      i               1.068966
Name: width, Length: 168, dtype: float64

## Шаг №5

С помощью нормировочных коэффициентов восстанавливаются прогнозируемые ширины символов в пикселях: для каждого символа `X` и шрифта `F` шириной становится ширина корзины которой принадлежит символ, умноженная на нормировочный коэффициент данного шрифта.

In [15]:
char_widths_calc_df = char_widths_s.to_frame().reset_index()\
    .merge(mean_order_df, on="char")\
    .merge(ncoeff_s.to_frame().reset_index(), \
           on=["font_face", "font_size", "font_version"], \
           suffixes=("_original", "_ncoeff"))\
    .assign(width_calc=lambda r: (np.round(r.cart_id.map(cart_widths) * r.width_ncoeff)).astype(int))
char_widths_calc_df.head()

,font_face,font_size,font_version,char,width_original,order,cart_id,width_ncoeff,width_calc
0,Courier,9,,,9,9,0,0.37931,4
1,Courier,9,,!,9,28,1,0.37931,7
2,Courier,9,,"""",9,28,1,0.37931,7
3,Courier,9,,#,9,75,2,0.37931,8
4,Courier,9,,$,9,75,2,0.37931,8


In [16]:
mean_width_df = char_widths_calc_df.groupby(["cart_id", "char"])[["width_original", "width_calc"]].mean()\
    .assign(width_diff=lambda r: r.width_calc - r.width_original).reset_index()
mean_width_df

,cart_id,char,width_original,width_calc,width_diff
0,0,,8.392857,7.434524,-0.958333
1,0,',8.095238,7.434524,-0.660714
2,0,(,11.577381,7.434524,-4.142857
3,0,),11.541667,7.434524,-4.107143
4,0,",",8.845238,7.434524,-1.410714
...,...,...,...,...,...
533,8,風,26.154762,26.410714,0.255952
534,8,高,26.154762,26.410714,0.255952
535,9,@,24.827381,27.077381,2.250000
536,9,W,26.517857,27.077381,0.559524


График ошибок: для каждого символа `X` каждой корзины рисуется разница между спрогнозированной шириной и фактической.

In [17]:
plot_matrix([
    ggplot(mean_width_df[mean_width_df.cart_id == cart_id], \
           aes(as_discrete("char", order_by="width_diff", order=1), "width_diff")) + \
    geom_bar(stat="identity", sampling=sampling_pick(538)) + \
    coord_flip() + ylim(-6, 6) + \
    ggtitle("cart_id = {0}".format(cart_id))
    for cart_id in range(carts_count - 1, -1, -1)
])

## Шаг №6

Второй `geom_boxplot()` с вариацией спрогнозированных ширин символов для каждой корзины.

In [18]:
ggplot(char_widths_calc_df, aes(as_discrete("cart_id", order=-1), "width_calc")) + geom_boxplot()